In [1]:
from collections import defaultdict
class Graph:
    def __init__(self, connections, directed):
        self.graph = defaultdict(set)
        self.directed = directed
        self.add_connections(connections)
        
    def add_connections(self, connections):
        for node1, node2 in connections:
            self.add(node1, node2)
                
    def add(self, node1, node2):
        self.graph[node1].add(node2)
        if not self.directed:
            self.graph[node2].add(node1)
    def remove(self, node):
        for source, targets in self.graph.items():
            try:
                targets.remove(node)
            except KeyError:
                pass
        try:
            del self.graph[node]
        except KeyError:
            pass
        
    def removeEdge(self, edge):
        source, target = edge
        try:
            self.graph[source].remove(target)
        except KeyError:
            pass
        if not self.directed:
            try:
                self.graph[target].remove(source)
            except KeyError:
                pass
        
    def nodes(self):
        nodes = set()
        for source, targets in self.graph.items():
            nodes.add(source)
            nodes.update(targets)
        return list(nodes)
    
    def edges(self, sourceSpecified=None):
        edges = []
        if sourceSpecified:
            for target in self.graph[sourceSpecified]:
                edges.append((sourceSpecified, target))
        else:
            for source in self.graph:
                for target in self.graph[source]:
                    edges.append((source, target))
                    if not self.directed:
                        try:
                            edges.remove((target, source))
                        except ValueError:
                            pass
        return edges

In [2]:
import unittest
class TestGraph(unittest.TestCase):
    def setUp(self):
        self.connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(self.connections, directed=True)
    
    def test_add_connections(self):
        newConnections = [('G', 'H'), ('G', 'A')]
        self.graph.add_connections(newConnections)
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 2)
        self.assertTrue('H' in gConnections and 'A' in gConnections)
        
    def test_add(self):
        self.graph.add('G','H')
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 1)
        self.assertTrue('H' in gConnections)
        
    def test_nodes(self):
        expectedNodes = set(['A','B','C','D','E','F','G'])
        nodes = self.graph.nodes()
        self.assertEqual(set(nodes), expectedNodes)
    def test_remove(self):
        self.graph.remove('B')
        aConnections = self.graph.graph['A']
        self.assertEqual(len(aConnections), 0)
        self.assertTrue('B' not in self.graph.graph)
        
    def test_removeEdge(self):
        self.graph.removeEdge(('A','B'))
        edgesFromA = self.graph.edges('A')
        self.assertEqual(edgesFromA, [])
    
    def test_edges(self):
        edges = self.graph.edges()
        for edge in edges:
            self.assertTrue(edge in self.connections)
        edgesFromA = self.graph.edges('A')
        self.assertEqual(edgesFromA, [('A','B')])
        
testSuiteGraph = unittest.TestLoader().loadTestsFromTestCase(TestGraph)
runner = unittest.TextTestRunner()
runner.run(testSuiteGraph)

......
----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [3]:
def breadthFirstSearch(graph, start):
    nodesAtLevel = [start]
    visited = []
    while len(nodesAtLevel) > 0:
        nodesAtNextLevel = []
        yield nodesAtLevel
        for node in nodesAtLevel:
            targets = graph.graph[node]
            for target in targets:
                if target not in visited:
                    nodesAtNextLevel.append(target)
            visited.append(node)
        nodesAtLevel = nodesAtNextLevel
        
        
def depthFirstSearch(graph, start):
    nodesToVisit = [start]
    visited = []
    while len(nodesToVisit) > 0:
        node = nodesToVisit.pop()
        yield node
        targets = graph.graph[node]
        for target in targets:
            if target not in visited:
                nodesToVisit.append(target)
        visited.append(node)

In [4]:
import unittest
from collections import Counter
class TestSearch(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_breadthFirstSearch(self):
        index = 0
        nodesAtLevels = [['A'],['B'],['C'],['D','F'],['E','G']]
        for nodes in breadthFirstSearch(self.graph, 'A'):
            self.assertEqual(Counter(nodes), Counter(nodesAtLevels[index]))
            index += 1
            
    def test_depthFirstSearch(self):
        visited = set()
        for node in depthFirstSearch(self.graph, 'A'):
            visited.add(node)
        self.assertEqual(len(visited), 7)
        
testSuiteSearch = unittest.TestLoader().loadTestsFromTestCase(TestSearch)
runner = unittest.TextTestRunner()
runner.run(testSuiteSearch)

..
----------------------------------------------------------------------
Ran 2 tests in 0.008s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [5]:
# 4.1 Route Between Nodes: Given a directed graph, design an algorithm to find out whether there is a route between two nodes.

# breadth first or depth first search starting from one node and looking for the other
# time complexity: O(V+E)
# space complexity: O(V)
        
def routeBetweenNodes(graph, start, end):
    for nodes in breadthFirstSearch(graph, start):
        if end in nodes:
            return True
    return False

In [6]:
import unittest
class TestRouteBetweenNodes(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_routeBetweenNodes(self):
        self.assertTrue(routeBetweenNodes(self.graph, 'A', 'G'))
        self.assertFalse(routeBetweenNodes(self.graph, 'D', 'B'))
        
testSuiteRouteBetweenNodes = unittest.TestLoader().loadTestsFromTestCase(TestRouteBetweenNodes)
runner = unittest.TextTestRunner()
runner.run(testSuiteRouteBetweenNodes)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [7]:
class BinaryTreeNode:
    def __init__(self, data):
        self.data = data
        self.parent = None
        self.leftChild = None
        self.rightChild = None
    def getLeftChild(self):
        return self.leftChild
    def getRightChild(self):
        return self.rightChild
    def setRightChild(self, node):
        self.rightChild = node
    def setLeftChild(self, node):
        self.leftChild = node
    def getData(self):
        return self.data
    def getParent(self):
        return self.parent
    def setParent(self, parent):
        self.parent = parent

In [8]:
import unittest
class TestBinaryTreeNode(unittest.TestCase):
    def setUp(self):
        self.node1 = BinaryTreeNode(1)
        self.node2 = BinaryTreeNode(2)
        self.node3 = BinaryTreeNode(3)
        self.node1.setLeftChild(self.node2)
        self.node2.setParent(self.node1)
    
    def test_getLeftChild(self):
        self.assertEqual(self.node1.getLeftChild(), self.node2)
        self.assertTrue(self.node2.getLeftChild() is None)
        
    def test_getRightChild(self):
        self.assertTrue(self.node1.getRightChild() is None)
        
    def test_setRightChild(self):
        self.node1.setRightChild(self.node3)
        self.assertEqual(self.node1.getRightChild().getData(), 3)
        
    def test_setLeftChild(self):
        self.node1.setLeftChild(self.node3)
        self.assertEqual(self.node1.getLeftChild().getData(), 3)
        
    def test_getData(self):
        self.assertEqual(self.node1.getData(), 1)
        
    def test_getParent(self):
        self.assertEqual(self.node2.getParent(), self.node1)
    
    def test_setParent(self):
        self.node3.setParent(self.node2)
        self.assertEqual(self.node3.getParent(), self.node2)
        
testSuiteBinaryTreeNode = unittest.TestLoader().loadTestsFromTestCase(TestBinaryTreeNode)
runner = unittest.TextTestRunner()
runner.run(testSuiteBinaryTreeNode)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.019s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

In [9]:
class BinaryTree:
    def __init__(self, data=[]):
        self.root = None
        for datum in data:
            self.add(datum)
    
    def add(self, data):
        newNode = BinaryTreeNode(data)
        
        if self.root:
            curNode = self.root
            while curNode:
                if data > curNode.getData():
                    if not curNode.getRightChild():
                        curNode.setRightChild(newNode)
                        newNode.setParent(curNode)
                        break
                    else:
                        curNode = curNode.getRightChild()
                else:
                    if not curNode.getLeftChild():
                        curNode.setLeftChild(newNode)
                        newNode.setParent(curNode)
                        break
                    else:
                        curNode = curNode.getLeftChild()
        else:
            self.root = newNode

    def search(self, data):
        return self.search_recursive(self.root, data)
        
    def search_recursive(self, node, data):
        if node:
            if node.getData() == data:
                return node
            if data > node.getData():
                return self.search_recursive(node.getRightChild(), data)
            else:
                return self.search_recursive(node.getLeftChild(), data)
        else:
            return None
    def inOrderTraversal(self, node, function):
        if node:
            self.inOrderTraversal(node.getLeftChild(), function)
            function(node.getData())
            self.inOrderTraversal(node.getRightChild(), function)
            
    def print(self):
        self.inOrderTraversal(self.root, print)
        
    def height(self):
        return self.height_recursive(self.root)
    def height_recursive(self, node):
        if not node:
            return 0
        leftHeight = self.height_recursive(node.getLeftChild())
        rightHeight = self.height_recursive(node.getRightChild())
        return max(leftHeight, rightHeight) + 1

In [10]:
import unittest
class TestBinaryTree(unittest.TestCase):
    def setUp(self):
        self.binaryTree = BinaryTree([5,4,6,3,7,1,8])
    def test_add(self):
        found = self.binaryTree.search(2)
        self.assertTrue(found is None)
        self.binaryTree.add(2)
        found = self.binaryTree.search(2)
        self.assertEqual(found.getData(), 2)
        
    def test_search(self):
        found = self.binaryTree.search(1)
        self.assertTrue(found is not None)
        self.assertEqual(found.getData(), 1)
        found = self.binaryTree.search(9)
        self.assertTrue(found is None)
        
    def test_height(self):
        self.assertEqual(self.binaryTree.height(), 4)
        self.binaryTree.add(0)
        self.assertEqual(self.binaryTree.height(), 5)
        
testSuiteBinaryTree = unittest.TestLoader().loadTestsFromTestCase(TestBinaryTree)
runner = unittest.TextTestRunner()
runner.run(testSuiteBinaryTree)

...
----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [11]:
# 4.2 Minimal Tree: Given a sorted (increasing order) array with unique integer elements, write an algorithm to create a binary search tree with minimal height.

# find middle element and add it to tree
# repeat with left and right half

def minimalTree(array):
    binarySearchTree = BinaryTree()
    minimalTree_recursive(binarySearchTree, array)
    return binarySearchTree
def minimalTree_recursive(tree, array):
    if len(array) == 0:
        return
    middleIndex = len(array) // 2
    tree.add(array[middleIndex])
    lowerLow = 0
    lowerHigh = middleIndex
    if lowerLow < lowerHigh:
        minimalTree_recursive(tree, array[lowerLow:lowerHigh])
    higherLow = middleIndex + 1
    higherHigh = len(array)
    if higherLow < higherHigh:
        minimalTree_recursive(tree, array[higherLow:higherHigh])

In [12]:
import unittest
from math import log
class TestMinimalTree(unittest.TestCase):
    def setUp(self):
        # array must be sorted in ascending order
        self.array = [3,4,6,7,8,14]
        
    def test_minimalTree(self):
        tree = minimalTree(self.array)
        expectedMinHeight = round(log(len(self.array), 2))
        self.assertEqual(tree.height(), expectedMinHeight)
        
testSuiteMinimalTree = unittest.TestLoader().loadTestsFromTestCase(TestMinimalTree)
runner = unittest.TextTestRunner()
runner.run(testSuiteMinimalTree)

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [13]:
class LinkedListNode:
    def __init__(self, value):
        self.value = value
        self.nextNode = None
    def getNextNode(self):
        return self.nextNode
    def setNextNode(self, node):
        self.nextNode = node
    def getValue(self):
        return self.value
    def setValue(self, value):
        self.value = value
        
class LinkedList:
    head = None
    def __init__(self, values):
        try:
            for value in reversed(values):
                self.add(value)
        except TypeError:
            if type(values) is LinkedListNode:
                self.head = values
            else:
                self.head = LinkedListNode(values)
    def add(self, data):
        newNode = LinkedListNode(data)
        newNode.setNextNode(self.head)
        self.head = newNode
    def get(self, index):
        node = self.head
        data = None
        for i in range(index):
            if node:
                node = node.getNextNode()
            else:
                break
        if node:
            data = node.getValue()
        return data
    def print(self):
        print(self.getList())
    def length(self):
        node = self.head
        length = 0
        while node:
            length += 1
            node = node.getNextNode()
        return length
    def getList(self):
        node = self.head
        nodes = []
        while node is not None:
            nodes.append(node.getValue())
            node = node.getNextNode()
        return nodes

In [14]:
# 4.3 List of Depths: Given a binary tree, design an algorithm which creates a linked list of all the nodes at each depth (e.g., if you have a tree with depth D, you'll have D linked lists)

# breadth first search from root, add nodes at each level to linked lists

def breadthFirstSearch_binary(tree):
    nodesAtLevel = [tree.root]
    while len(nodesAtLevel) > 0:
        nodesAtNextLevel = []
        yield nodesAtLevel
        for node in nodesAtLevel:
            if node:
                leftChild = node.getLeftChild()
                rightChild = node.getRightChild()
                if leftChild:
                    nodesAtNextLevel.append(leftChild)
                if rightChild:
                    nodesAtNextLevel.append(rightChild)
            
        nodesAtLevel = nodesAtNextLevel
        
def listOfDepths(tree):
    listOfLists = []
    for nodes in breadthFirstSearch_binary(tree):
        listOfLists.append(LinkedList([n.getData() for n in nodes]))
    return listOfLists

In [15]:
import unittest
from math import log
from collections import Counter
class TestListOfDepths(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.expectedArray = [[7],[4,14],[3,6,8]]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_listOfDepths(self):
        lists = listOfDepths(self.tree)
        lists = [l.getList() for l in lists]
        for l, exp in zip(lists, self.expectedArray):
            self.assertEqual(Counter(l), Counter(exp))
        
testSuiteListOfDepths = unittest.TestLoader().loadTestsFromTestCase(TestListOfDepths)
runner = unittest.TextTestRunner()
runner.run(testSuiteListOfDepths)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [16]:
# 4.4 Check Balanced: Implement a function to check if a binary tree is balanced. For the purposes of the question, a balanced tree is defined to be a tree such that the heights of the two subtrees of any node never differ by more than one.

# recursively compare heights of left and right subtrees 
# if difference is bigger than 1, false

def checkBalanced(tree):
    height, balanced = checkBalanced_recursive(tree.root)
    return balanced
# returns (height, balanced)
def checkBalanced_recursive(node):
    if not node:
        return (0, True)
    leftHeight, leftBalanced = checkBalanced_recursive(node.getLeftChild())
    rightHeight, rightBalanced = checkBalanced_recursive(node.getRightChild())
    balanced = leftBalanced and rightBalanced
    if abs(leftHeight - rightHeight) > 1:
        balanced = False
    return (max(leftHeight, rightHeight) + 1, balanced)

In [17]:
import unittest
class TestCheckBalanced(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.expectedArray = [[7],[4,14],[3,6,8]]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_checkBalanced(self):
        self.assertTrue(checkBalanced(self.tree))
        self.tree.add(1)
        self.tree.add(0)
        self.assertFalse(checkBalanced(self.tree))
        
testSuiteCheckBalanced = unittest.TestLoader().loadTestsFromTestCase(TestCheckBalanced)
runner = unittest.TextTestRunner()
runner.run(testSuiteCheckBalanced)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [18]:
# 4.5 Validate BST: Implement a function to check if a binary tree is a binary search tree.

# check if max of left subtree is less than or equal to current node's data and min of right subtree is bigger than current n
# recursively check for left and right
# return whether valid bst, min and max of tree

def validateBST(tree):
    valid, minimum, maximum = validateBST_recursive(tree.root)
    return valid
def validateBST_recursive(node):
    if not node:
        return (True, None, None)
    leftValidBST, leftMin, leftMax = validateBST_recursive(node.getLeftChild())
    rightValidBST, rightMin, rightMax = validateBST_recursive(node.getRightChild())
    valid = True
    
    minimum = leftMin
    if leftMin is None:
        minimum = node.getData()
    maximum = rightMax
    if rightMax is None:
        maximum = node.getData()
    if leftMax is not None:
        if leftMax > node.getData():
            valid = False
    if rightMin is not None:
        if rightMin < node.getData():
            valid = False
            
    return (valid, minimum, maximum)

In [19]:
import unittest
class TestValidateBST(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_validateBST(self):
        self.assertTrue(validateBST(self.tree))
        node = self.tree.root
        while node.getRightChild():
            node = node.getRightChild()
        newNode = BinaryTreeNode(0)
        node.setLeftChild(newNode)
        self.assertFalse(validateBST(self.tree))
        
testSuiteValidateBST = unittest.TestLoader().loadTestsFromTestCase(TestValidateBST)
runner = unittest.TextTestRunner()
runner.run(testSuiteValidateBST)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [20]:
# 4.6 Successor: Write an algorithm to find the "next" node (i.e., in-order successor) of a given node in a binary search tree. You may assume that each node has a link to its parent.

# next one should be left most node in right subtree
# if right child is null, can also look at left most node of parent's parent's right subtree
# the second case only works if the node's parent is the left child of the node's grandparent

def successor(node):
    rightChild = node.getRightChild()
    successorNode = None
    if rightChild:
        node = rightChild
        while node.getLeftChild():
            node = node.getLeftChild()
        successorNode = node
    else:
        parent = node.getParent()
        if parent:
            if parent.getLeftChild() == node:
                successorNode = parent
            else:
                grandparent = parent.getParent()
                if grandparent and grandparent.getLeftChild() == parent:
                    node = grandparent.getRightChild()
                    while node.getLeftChild():
                        node = node.getLeftChild()
                    successorNode = node
    return successorNode

In [21]:
import unittest
class TestSuccessor(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_successor(self):
        node = self.tree.root
        self.assertEqual(successor(node).getData(), 8)
        node = node.getLeftChild()
        self.assertEqual(successor(node).getData(), 6)
        node = node.getLeftChild()
        self.assertEqual(successor(node).getData(), 4)
        node = node.getParent().getRightChild()
        self.assertEqual(successor(node).getData(), 8)
        
testSuiteSuccessor = unittest.TestLoader().loadTestsFromTestCase(TestSuccessor)
runner = unittest.TextTestRunner()
runner.run(testSuiteSuccessor)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [22]:
# 4.7 Build Order: You are given a list of projects and a list of dependencies (which is a list of pairs of projects, where the first project is dependent on the second project). All of a project's dependecies must be built before the project is. Find a build order that will allow the projects to be built. If there is no valid build order, return an error.

# Example 
# Input: 
# projects: a,b,c,d,e,f
# dependencies: (d,a), (b,f), (d,b), (a,f), (c,d)
# Output: f,e,a,b,d,c

# topological sort
# source: https://en.wikipedia.org/wiki/Topological_sorting
# Kahn's algorithm
# set S of nodes with no incoming edges
# array L empty to contain nodes in topological order
# while S is not empty
#    remove node n from S and add it to end of L
#    for each edge from node n to node m
#        remove edge from graph
#        if node m has no incoming edges, add to S
# if graph still has edges
#    graph has at least one cycle
# else
#    return L (topologically sorted)

def topologicalSort(graph):
    sortedNodes = []
    
    edges = graph.edges()
    sourceNodes = graph.nodes()
    for _, target in edges:
        try:
            sourceNodes.remove(target)
        except ValueError:
            pass
    while len(sourceNodes) > 0:
        node = sourceNodes.pop()
        sortedNodes.append(node)
        edgesFromNode = graph.edges(node)
        for edge in edgesFromNode:
            _, target = edge
            graph.removeEdge(edge)
            edges = graph.edges()
            targetHasIncomingEdges = False
            for _, tempTarget in edges:
                if tempTarget == target:
                    targetHasIncomingEdges = True
                    break
            if not targetHasIncomingEdges:
                sourceNodes.append(target)
    edges = graph.edges()
    if len(edges) > 0:
        raise ValueError('graph has at least one cycle')
    else:
        return sortedNodes

def buildOrder(projects, dependencies):
    graph = Graph(dependencies, directed=True)
    sortedNodes = topologicalSort(graph)
    for project in projects:
        if project not in sortedNodes:
            sortedNodes.append(project)
            #independent projects can be started immediately
    return list(reversed(sortedNodes))

In [23]:
import unittest
class TestBuildOrder(unittest.TestCase):
    def setUp(self):
        self.dependencies = [('d','a'), ('b','f'), ('d','b'), ('a','f'), ('c','d')]
        self.projects = ['a','b','c','d','e','f']    
        
    def test_buildOrder(self):
        sortedProjects = buildOrder(self.projects, self.dependencies)
        for dependency in self.dependencies:
            dependent, independent = dependency
            indexOfDependent = sortedProjects.index(dependent)
            indexOfIndependent = sortedProjects.index(independent)
            self.assertTrue(indexOfIndependent < indexOfDependent)
        for project in self.projects:
            self.assertTrue(project in sortedProjects)
        
testSuiteBuildOrder = unittest.TestLoader().loadTestsFromTestCase(TestBuildOrder)
runner = unittest.TextTestRunner()
runner.run(testSuiteBuildOrder)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [24]:
# 4.8 First Common Ancestor: Design an algorithm and write code to find the first common ancestor of two nodes in a binary tree. Avoid storing additional nodes in a data structure. NOTE: This is not necessarily a binary search tree.

# if nodes have pointers to parents, this problem would be the same as a previous problem to find the intersection of two linked lists
# let's do this without pointers to parents
# search binary tree for node1 and node2
# if node1 and node2 are in different subtrees at a single node, that node is the common ancestor
# return node1 if node1 is contained in subtree
# return node2 if node2 is contained in subtree
# return commonAncestor if node1 and node 2 are in different subtrees or if subtree has commonAncestor
# return None if subtree doesn't have node1, node2, or the common ancestor
# time complexity: O(n)

def firstCommonAncestor(tree, node1, node2):
    node, ancestorFound = firstCommonAncestor_recursive(tree.root, node1, node2)
    if ancestorFound:
        return node
    else:
        return None

def firstCommonAncestor_recursive(currentNode, node1, node2):
    if currentNode is None:
        return (None, False)
    data = currentNode.getData()
    leftResult, ancestorFoundLeft = firstCommonAncestor_recursive(currentNode.getLeftChild(), node1, node2)
    rightResult, ancestorFoundRight = firstCommonAncestor_recursive(currentNode.getRightChild(), node1, node2)
    
    if ancestorFoundRight:
        #ancestor found from right
        return (rightResult, True)
    if ancestorFoundLeft:
        return (leftResult, True)
    
    node1Found = data == node1 or leftResult == node1 or rightResult == node2
    node2Found = data == node2 or leftResult == node2 or rightResult == node2
    
    if node1Found and node2Found:
        return (data, True)
    if node1Found:
        return (node1, False)
    if node2Found:
        return (node2, False)
    return (None, False)

In [25]:
import unittest
class TestFirstCommonAncestor(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
            
    def test_firstCommonAncestor(self):
        self.assertEqual(firstCommonAncestor(self.tree, 3, 6), 4)
        self.assertEqual(firstCommonAncestor(self.tree, 3, 4), 4)
        self.assertEqual(firstCommonAncestor(self.tree, 6, 8), 7)
        self.assertTrue(firstCommonAncestor(self.tree, 16, 7) is None)
        
testSuiteFirstCommonAncestor = unittest.TestLoader().loadTestsFromTestCase(TestFirstCommonAncestor)
runner = unittest.TextTestRunner()
runner.run(testSuiteFirstCommonAncestor)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [26]:
# 4.9 BST Sequences: A binary search tree was created by traversing through an arry from left to right and inserting each element. Given a binary search tree with distinct elements, print all possible arrays that could have led to this tree.
# Example
# Input:
#    2->3
#    |
#    v
#    1
# Output: (2, 1, 3), (2, 3, 1)

# parents must be added to the binary search tree before their children
# the order of nodes at the same level do not matter because of how insertion works in a binary search tree
# thinking recursively, we treat each node as a root and combine bst sequences from children in all different orders preserving order within their subtrees and prepend the root at the end

def bstSequences(tree):
    return bstSequences_recursive(tree.root)
    
def bstSequences_recursive(node):
    if node is None:
        return [[]]
    
    bstSequencesLeft = bstSequences_recursive(node.getLeftChild())
    bstSequencesRight = bstSequences_recursive(node.getRightChild())
    
    weaved = []
    for bstLeft in bstSequencesLeft:
        for bstRight in bstSequencesRight:
            weave(bstLeft, bstRight, [node.getData()], weaved)
    return weaved
    
# combine two lists with a specified prefix preserving order within lists 
def weave(first, second, prefix, weaved):
    if len(first) == 0 or len(second) == 0:
        result = prefix + first + second
        weaved.append(result)
        return
        
    first1 = list(first)
    prefix1 = list(prefix)
    prefix1.append(first1.pop(0))
    weave(first1, second, prefix1, weaved)
    
    second1 = list(second)
    prefix2 = list(prefix)
    prefix2.append(second1.pop(0))
    weave(first, second1, prefix2, weaved)

In [27]:
import unittest
from operator import eq
from math import factorial

# source: https://stackoverflow.com/questions/17119116/how-many-ways-can-you-insert-a-series-of-values-into-a-bst-to-form-a-specific-tr?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
def choose(m, n):
    return factorial(m) / (factorial(m-n) * factorial(n))

def numElementsIn(node):
    if node is None:
        return 0
    numNodesInLeft = numElementsIn(node.getLeftChild())
    numNodesInRight = numElementsIn(node.getRightChild())
    return numNodesInLeft + numNodesInRight + 1

def countInsertionOrderings(tree):
    if tree is None:
        return 1
    m = numElementsIn(tree.getRightChild())
    n = numElementsIn(tree.getLeftChild())
    return choose(m+n, n) * countInsertionOrderings(tree.getLeftChild()) * countInsertionOrderings(tree.getRightChild())

class TestBSTSequences(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
    
    def test_bstSequences(self):
        sequences = bstSequences(self.tree)
        # uniqueness of sequences
        for i in range(len(sequences)-1):
            for j in range(i + 1, len(sequences)):
                self.assertTrue(sequences[i] != sequences[j])
        for seq in sequences:
            self.assertEqual(Counter(self.array), Counter(seq))
        expectedCount = countInsertionOrderings(self.tree.root)
        self.assertEqual(expectedCount, len(sequences))
        
testSuiteBSTSequences = unittest.TestLoader().loadTestsFromTestCase(TestBSTSequences)
runner = unittest.TextTestRunner()
runner.run(testSuiteBSTSequences)

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [28]:
# 4.10 Check Subtree: T1 and T2 are two very large binary trees, with T1 much bigger than T2. Create an alorithm to determine if T2 is a subtree of T1.
# A tree T2 is a subtree of T1 if there exists a node n in T1 such that the subtree of n is identical to T2. That is, if you cut off the tree at node n, the two trees would be identical.

# search T1 for the root of T2
# if node matches root, then try to match subtrees

def checkSubtree(t1, t2):
    if t1 is None:
        return t2 is None
    if t2 is None:
        return False
    if t1.getData() == t2.getData():
        if matchTrees(t1, t2):
            return True
    return (checkSubtree(t1.getLeftChild(), t2) or checkSubtree(t1.getRightChild(), t2))
def matchTrees(t1, t2):
    if t1 is None and t2 is None:
        return True
    if t1 is None or t2 is None:
        return False
    if t1.getData() != t2.getData():
        return False
    return matchTrees(t1.getLeftChild(), t2.getLeftChild()) and matchTrees(t1.getRightChild(), t2.getRightChild())

array = [7,4,14,3,6,8]
tree = BinaryTree()
for elem in array:
    tree.add(elem)
    
array2 = [3,6]
tree2 = BinaryTree()
for elem in array2:
    tree2.add(elem)
res = checkSubtree(tree.root, tree2.root)
print(res)

False


In [29]:
import unittest
class TestCheckSubtree(unittest.TestCase):
    def setUp(self):
        self.array = [7,4,14,3,6,8]
        self.tree = BinaryTree()
        for elem in self.array:
            self.tree.add(elem)
        
        self.array1 = [3,6]
        self.subtree1 = BinaryTree()
        for elem in self.array1:
            self.subtree1.add(elem)
            
        self.array2 = [4,3,6]
        self.subtree2 = BinaryTree()
        for elem in self.array2:
            self.subtree2.add(elem)
            
    def test_checkSubtree(self):
        self.assertTrue(checkSubtree(self.tree.root, self.subtree2.root))
        self.assertFalse(checkSubtree(self.tree.root, self.subtree1.root))
        
testSuiteCheckSubtree = unittest.TestLoader().loadTestsFromTestCase(TestCheckSubtree)
runner = unittest.TextTestRunner()
runner.run(testSuiteCheckSubtree)

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [30]:
# 4.11 Random Node: You are implementing a binary tree class from scratch which, in addition to insert, find, and delete, has a method getRandomNode() which returns a random node from the tree. All nodes should be equally likely to be chosen. Design and implement an algorithm for getRandomNode, and explain how you would implement the rest of the methods.

# pick current node with probability of 1/n
# pick left with left size / n
# pick right with right size / n
# nodes can store sizes of left and right subtrees
# adjust on insertions and deletes

import random
class RBSTNode:
    def __init__(self, value=None):
        self.value = value
        self.size = 1
        self.leftChild = None
        self.rightChild = None
    def setValue(self, newValue):
        self.value = newValue
    def getValue(self):
        return self.value
    def getSize(self):
        return self.size
    def setSize(self, size):
        self.size = size
    def hasLeftChild(self):
        return (self.leftChild is not None)
    def hasRightChild(self):
        return (self.rightChild is not None)
    def getLeftChild(self):
        return self.leftChild
    def getRightChild(self):
        return self.rightChild
    def setLeftChild(self, leftChild):
        self.leftChild = leftChild
    def setRightChild(self, rightChild):
        self.rightChild = rightChild

In [31]:
import unittest
class TestRBSTNode(unittest.TestCase):
    def setUp(self):
        self.node1 = RBSTNode(5)
        self.node2 = RBSTNode(10)
            
    def test_setValue(self):
        self.node1.setValue(6)
        self.assertEqual(self.node1.getValue(), 6)
        
    def test_getValue(self):
        self.assertEqual(self.node1.getValue(), 5)
        
    def test_getSize(self):
        self.assertEqual(self.node1.getSize(), 1)
        
    def test_setSize(self):
        self.node1.setSize(2)
        self.assertEqual(self.node1.getSize(), 2)
        
    def test_hasLeftChild(self):
        self.node1.setLeftChild(self.node2)
        self.assertTrue(self.node1.hasLeftChild())
        
    def test_hasRightChild(self):
        self.node2.setRightChild(self.node1)
        self.assertTrue(self.node2.hasRightChild())
        
    def test_getLeftChild(self):
        self.node2.setLeftChild(self.node1)
        self.assertEqual(self.node2.getLeftChild(), self.node1)
        
    def test_getRightChild(self):
        self.node1.setRightChild(self.node2)
        self.assertEqual(self.node1.getRightChild(), self.node2)
        
    def test_setLeftChild(self):
        self.node1.setLeftChild(self.node2)
        self.assertEqual(self.node1.getLeftChild(), self.node2)
        
    def test_setRightChild(self):
        self.node2.setRightChild(self.node1)
        self.assertEqual(self.node2.getRightChild(), self.node1)
        
testSuiteRBSTNode = unittest.TestLoader().loadTestsFromTestCase(TestRBSTNode)
runner = unittest.TextTestRunner()
runner.run(testSuiteRBSTNode)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.029s

OK


<unittest.runner.TextTestResult run=10 errors=0 failures=0>

In [32]:
class RandomBinarySearchTree:
    def __init__(self, values=[], seed=0):
        random.seed(seed)
        self.root = None
        for value in values:
            self.insert(value)
            
    def size(self):
        return self.root.getSize()
        
    def insert(self, value):
        newNode = RBSTNode(value)
        if self.root is None:
            self.root = newNode
            return
        curNode = self.root
        while curNode:
            curNode.setSize(curNode.getSize() + 1)
            if value <= curNode.getValue():
                nextNode = curNode.getLeftChild()
                
            else:
                nextNode = curNode.getRightChild()
            if nextNode is None:
                break
            curNode = nextNode
        if value <= curNode.getValue():
            curNode.setLeftChild(newNode)
        else:
            curNode.setRightChild(newNode)
        
    def find(self, value, function=lambda x: x):
        curNode = self.root
        nodesTouched = []
        while curNode:
            if curNode.getValue() == value:
                for node in nodesTouched:
                    node.setSize(function(node.getSize()))
                return curNode
            nodesTouched.append(curNode)
            if value <= curNode.getValue():
                curNode = curNode.getLeftChild()
            else:
                curNode = curNode.getRightChild()
        return None
    def numChildren(self, node):
        numChildren = 0
        if node.hasLeftChild():
            numChildren += 1
        if node.hasRightChild():
            numChildren += 1
        return numChildren
    def delete(self, value):
        nodeToDelete = self.find(value, lambda x: x-1)
        self.deleteNode(nodeToDelete)
    
    def parent(self, node):
        if self.root is None or node is None or node == self.root:
            # tree is empty or no parent
            return None
        parent = self.root
        value = node.getValue()
        child = parent.getLeftChild() if value <= parent.getValue() else parent.getRightChild()
        if child == node:
            return parent
        while child and child != node:
            parent = child
            if value <= parent.getValue():
                child = parent.getLeftChild()
            else:
                child = parent.getRightChild()
        if child:
            return parent
        else:
            # node not found in tree
            return None
                
    def deleteNode(self, node):
        if node is None:
            return
        # count children
        nChildren = self.numChildren(node)
        if nChildren == 0:
            parent = self.parent(node)
            if parent:
                if parent.getLeftChild() == node:
                    parent.setLeftChild(None)
                elif parent.getRightChild() == node:
                    parent.setRightChild(None)
                else:
                    raise ValueError('node is not a child of parent')
            elif node == self.root:
                #last node in tree
                self.root = None
        elif nChildren == 1:
            node.setSize(node.getSize() - 1)
            if node.hasLeftChild():
                nodeToDelete = node.getLeftChild()
            else:
                nodeToDelete = node.getRightChild()
            newValue = nodeToDelete.getValue()
            self.deleteNode(nodeToDelete)
            node.setValue(newValue)
            
        elif nChildren == 2:
            # copy in order successor if it exists and delete in order successor 
            node.setSize(node.getSize() - 1)
            successor = node.getRightChild()
            while successor.hasLeftChild():
                successor.setSize(successor.getSize() - 1)
                successor = successor.getLeftChild()
            newValue = successor.getValue()
            self.deleteNode(successor)
            node.setValue(newValue)
            
    def getRandomNode(self):
        #chance of picking current node should be 1/n where n is the size of the subtree
        #picking left side should be left size * 1/n
        # can be improved by not repicking random number
        node = self.root
        randomInt = random.randint(0, node.getSize() - 1)
        while True:
            if node is None:
                return None
            leftSize = 0 if not node.hasLeftChild() else node.getLeftChild().getSize()
            if randomInt < leftSize:
                node = node.getLeftChild()
            elif randomInt == leftSize:
                return node
            else:
                randomInt -= leftSize + 1
                node = node.getRightChild()
            
    def print_recursive(self, node):
        if node is None:
            return
        self.print_recursive(node.getLeftChild())
        print(node.getValue())
        self.print_recursive(node.getRightChild())
    def print(self):
        self.print_recursive(self.root)

In [33]:
import unittest
class TestRandomBinarySearchTree(unittest.TestCase):
    def setUp(self):
        values = [5,3,8,2,10,1]
        self.rbst = RandomBinarySearchTree(values)
        
    def test_size(self):
        self.assertEqual(self.rbst.size(), 6)
        
    def test_insert(self):
        self.rbst.insert(11)
        self.assertTrue(self.rbst.size() == 7 and self.rbst.find(11) is not None)
        
    def test_delete(self):
        self.rbst.delete(5)
        self.assertTrue(self.rbst.find(5) is None and self.rbst.size() == 5)
        
    def test_find(self):
        node = self.rbst.find(1)
        self.assertTrue(node is not None and node.getValue() == 1)
        
    def test_getRandomNode(self):
        for i in range(10):
            node = self.rbst.getRandomNode()
            self.assertTrue(self.rbst.find(node.getValue()) is not None)
        
testSuiteRandomBinarySearchTree = unittest.TestLoader().loadTestsFromTestCase(TestRandomBinarySearchTree)
runner = unittest.TextTestRunner()
runner.run(testSuiteRandomBinarySearchTree)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.011s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [39]:
# 4.12 Paths with Sum: You are given a binary search tree in which each node contains an integer value (which might be positive or negative).
# Design an algorithm to count the number of paths that sum to a given value. The path does not need to start or end at the root or a leaf,
# but it must go downwards (traveling only from parent nodes to child nodes).

# can think about this problem as finding the number of consecutive subsets of a path from root to leaf that add to target sum
# can store a running sum to find starting points for the subsets
# a subset sums to a target sum when the current running sum - target sum is a running sum seen already
# 4 -> 5 -> 7  -> -4 -> -3
# 4    9    16    12    9

def pathsWithSum(tree, targetSum):
    previousRunningSums = {}
    return pathsWithSum_recursive(tree.root, 0, targetSum, previousRunningSums)
def pathsWithSum_recursive(node, runningSum, targetSum, previousRunningSums):
    if node is None:
        return 0
    numPaths = 0
    runningSum += node.getData()
    previousSum = runningSum - targetSum
    if previousSum == 0:
        numPaths += 1
    if previousSum in previousRunningSums:
        numPaths += previousRunningSums[previousSum]
    if runningSum in previousRunningSums:
        previousRunningSums[runningSum] += 1
    else:
        previousRunningSums[runningSum] = 1
    numPaths += pathsWithSum_recursive(node.getLeftChild(), runningSum, targetSum, previousRunningSums)
    numPaths += pathsWithSum_recursive(node.getRightChild(), runningSum, targetSum, previousRunningSums)
    
    # remove runningsum from previousrunningsums
    if previousRunningSums[runningSum] > 0:
        previousRunningSums[runningSum] -= 1
    else:
        del previousRunningSums[runningSum]
    return numPaths

In [40]:
import unittest
class TestPathsWithSum(unittest.TestCase):
    def setUp(self):
        values = [1,-1,4,-2,0,2,3]
        self.binaryTree = BinaryTree(values)
        
    def test_pathsWithSum_multiple(self):
        self.assertEqual(pathsWithSum(self.binaryTree, 0), 3)
        
    def test_pathsWithSum_one(self):
        self.assertEqual(pathsWithSum(self.binaryTree, 6), 1)
        
    def test_pathsWithSum_zero(self):
        self.assertEqual(pathsWithSum(self.binaryTree, -5), 0)
        
testSuitePathsWithSum = unittest.TestLoader().loadTestsFromTestCase(TestPathsWithSum)
runner = unittest.TextTestRunner()
runner.run(testSuitePathsWithSum)

...
----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [41]:
# source: https://stackoverflow.com/questions/1732438/how-do-i-run-all-python-unit-tests-in-a-directory
testmodules = [
    TestGraph,
    TestSearch,
    TestRouteBetweenNodes, 
    TestBinaryTreeNode,
    TestBinaryTree,
    TestMinimalTree,
    TestListOfDepths,
    TestCheckBalanced,
    TestValidateBST,
    TestSuccessor,
    TestBuildOrder,
    TestFirstCommonAncestor,
    TestBSTSequences,
    TestCheckSubtree,
    TestRBSTNode,
    TestRandomBinarySearchTree,
    TestPathsWithSum
    ]

suite = unittest.TestSuite()

for t in testmodules:
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(t))

unittest.TextTestRunner().run(suite)

..............................................
----------------------------------------------------------------------
Ran 46 tests in 0.082s

OK


<unittest.runner.TextTestResult run=46 errors=0 failures=0>